In [ ]:
import os
from pathlib import Path
from pprint import pprint

import pandas as pd
from pyodm import Node

import seabeepy as sb

# Check mission progress

Checks processing status of missions submitted to the SeaBee platform.

## 1. Tasks on NodeODM

In [ ]:
node = Node.from_url("http://nodeodm")
print(node.info())

nodeodm_task_ids = node.get("task/list")
task_list = [node.get_task(t["uuid"]) for t in nodeodm_task_ids]
print("Total number of tasks:", len(task_list))

task_df = pd.DataFrame(
    [
        (
            idx,
            t.info().uuid,
            os.path.split(t.info().name)[1],
            t.info().images_count,
            t.info().progress,
            str(t.info().status).split(".")[1],
            t.output()[-1] if t.output() else "no message",
        )
        for idx, t in enumerate(task_list)
    ],
    columns=[
        "task_index",
        "uuid",
        "mission_name",
        "n_images",
        "progress",
        "status",
        "last_message",
    ],
)
task_df

### Task output

Check output of running tasks, could perhaps just look at one task since the output is quite big.

In [ ]:
for task in task_list:
    if str(task.info().status) == "TaskStatus.FAILED":
        print(f"Task {task.info().name} at {task.info().progress}%")
        for l in task.output()[-30:]:
            print(l)
        print("\n")

### Restart failed tasks

Restart task that was previously canceled, that had failed to process or that successfully completed. We just restart failed tasks.

In [ ]:
failed_tasks = [t for t in task_list if str(t.info().status) == "TaskStatus.FAILED"]
set([t.info().last_error for t in failed_tasks])

In [ ]:
# for task in failed_tasks:
#     if task.info().last_error == "Cannot process dataset":
#         task.restart()

In [ ]:
# print(task_list[0].info())
# sorted([n.name for n in node.options()])
# task_list[0].cancel()
# task_list[0].remove()

## 2. Folder structure

In [ ]:
# User input
base_dir = r"/home/notebook/shared-seabee-ns9879k/seabirds/2023/"

In [ ]:
%%capture

# Check mission data in 'base_dir'

dir_list = [f.parent for f in Path(base_dir).rglob("config.seabee.yaml")]

valid_configs = [f for f in dir_list if sb.ortho.check_config_valid(f)]
invalid_configs = [f for f in dir_list if not sb.ortho.check_config_valid(f)]

images = [f for f in dir_list if sb.ortho.check_subdir_exists(f, "images")]
no_images = [f for f in dir_list if not sb.ortho.check_subdir_exists(f, "images")]

orthos = [f for f in dir_list if sb.ortho.check_subdir_exists(f, "orthophoto")]
no_orthos = [f for f in dir_list if not sb.ortho.check_subdir_exists(f, "orthophoto")]

images_configs = [f for f in valid_configs if f in images]
valid_counts = [f for f in images_configs if sb.ortho.check_file_count(f)]
invalid_counts = [f for f in images_configs if not sb.ortho.check_file_count(f)]

to_process = [
    f
    for f in valid_counts
    if not sb.ortho.check_subdir_exists(f, "orthophoto")
]

no_mosaic = [f for f in to_process if not sb.ortho.parse_config(f)["mosaic"]]
no_publish = [f for f in to_process if not sb.ortho.parse_config(f)["publish"]]

In [ ]:
print("Scanning", base_dir)
print(len(dir_list), "sub-directories contain 'config.seabee.yaml' files.")
print(
    len(valid_configs),
    f"directories have valid config. files ({len(invalid_configs)} are invalid).",
)
print(
    len(images), f"directories have 'images' sub-directories ({len(no_images)} do not)."
)
print(
    len(orthos),
    f"directories have 'orthophoto' sub-directories ({len(no_orthos)} do not).",
)
print(
    len(images_configs),
    f"directories have both images and valid configs. {len(valid_counts)} of these have the correct number of images and {len(invalid_counts)} do not.",
)
print(
    len(to_process),
    f"missions can be processed ({len(no_mosaic)} have 'mosaic=False' and {len(no_publish)} have 'publish=False').",
)

print(f"\nThe following {len(invalid_configs)} folders have invalid config.:")
pprint(invalid_configs)

print(f"\nThe following {len(no_images)} folders have no images:")
pprint(no_images)

print(f"\nThe following {len(invalid_counts)} folders have invalid image counts:")
pprint(invalid_counts)

print(f"\nThe following {len(to_process)} folders are ready to be processed:")
pprint(to_process)

print(f"\nOf these, the following {len(no_mosaic)} folders have 'mosaic=False':")
pprint(no_mosaic)

print(f"\nand the following {len(no_publish)} folders have 'publish=False':")
pprint(no_publish)

In [ ]:
# Missions with non-standard ODM settings
[f for f in valid_configs if sb.ortho.parse_config(f).get("odm_options")]